# Let's get random!

### Running experiments with random operators, randomly picked base measurement ERP pairs, randomly generated sizes and layers

In [35]:
# first some general imports:
%matplotlib inline
import matplotlib, qiskit, pprint, itertools
from matplotlib import pyplot as plt
import numpy as np
from numpy import pi
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, schedule, assemble
from qiskit.circuit import *
from qiskit.circuit.library.standard_gates import *
from qiskit.providers.aer import QasmSimulator
from qiskit.test.mock import FakeAlmaden, FakeMelbourne
from qiskit.visualization import *
from qiskit.circuit.library import *
# load IBM Q account:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

ibmqfactory.load_account:WARNING:2021-06-04 03:08:31,378: Credentials are already in use. The existing account in the session will be replaced.


## Function to run experiments

In [36]:
from QSVT_utils import *
import json

def run_random_operator_experiment(n_min, n_max, datafolder, num_of_experiments, backend, experiment_id):
    
    datadict = {}
    
    if n_min < 2:
        print('Two is the absolute minimum for number of qubits in scrambler.')
        
    elif n_max <= n_min:
        print('The maximum n for number of qubits in scrambler must be bigger.')
        
    else:
        
        for expnum in range(num_of_experiments):
            
            id_num = str(expnum)
            npy_path = datafolder + id_num + ".npy"
            
            datadict[id_num] = {"op_npy":npy_path}

            n = np.random.randint(n_min, n_max)
            operator_num = np.random.randint(3, n_max)
            qnum = 2*n +1
            
            datadict[id_num]['qnum'] = qnum
            
            fst_half, sec_half = fst_n_sec_half_nums(qnum)
            bell_pairs = get_bell_pairs(qnum)
            measpair_inum = np.random.randint(0, len(bell_pairs)-1)
            meas_pairs = [x for x in bell_pairs[:-1]]+[[0,qnum-2]] 
            inum_alice, inum_bob = 0, qnum-1 
            basemeas = meas_pairs[measpair_inum] 
            
            from qiskit.aqua.utils import random_unitary
            random_operator_list = [get_random_unitary_operator(n) for x in range(operator_num)] 

            scrambling_circ = QuantumCircuit(qnum,qnum)
            for randomop in random_operator_list:
                scrambling_circ.append(randomop,fst_half)
                scrambling_circ.append(randomop.transpose(), sec_half)
            
            scrambling_op = Operator(scrambling_circ)
            
            with open(npy_path, 'wb') as f:
                np.save(f, scrambling_op.data)

            testcirc = QuantumCircuit(qnum,qnum) # new test circ
            entangle_bell_pairs(testcirc, bell_pairs) # initialize bell pairs
            testcirc.x(inum_alice) # initialize alice qubit in |1> state
            testcirc.append(scrambling_op, [x for x in range(qnum)]) # apply the scrambler 
            disentangle_bell_pair(testcirc, basemeas) # disentangle base meas bell pair
            apply_bob_gates(testcirc,inum_bob,basemeas) # apply decoding from measurement
            testcirc.measure(inum_bob,inum_bob) # measure bob qubit
            t_qc = transpile(testcirc, backend)
            shots = 1000
            qobj = assemble(t_qc, shots=shots)
            result = backend.run(qobj).result()
            fid_dict = {"1": sum([v for (k,v) in result.get_counts().items() if k[0]== "1"]),
                        "0": sum([v for (k,v) in result.get_counts().items() if k[0]== "0"])}
            fid = fid_dict.get("1")/shots
            
            datadict[id_num]['fidelity'] = fid
            datadict[id_num]['fid_dict'] = fid_dict

            s = str(scrambling_op.data)
            op_bits = []
            for c in s:
                bits = bin(ord(c))[2:]
                bits = '00000000'[len(bits):] + bits
                op_bits.extend([int(b) for b in bits])

            from pybdm import BDM
            bdm = BDM(ndim=1, nsymbols=2)
            bdm_val = bdm.bdm(np.array(op_bits)) # get bdm
            datadict[id_num]["bdm_val"] = bdm_val # update datadict
            bdm_score = bdm.nbdm(np.array(op_bits)) # bdm score
            datadict[id_num]["bdm_score"] = bdm_score # update datadict
            
    with open('.\data\datadict'+experiment_id+'.json', 'w') as f:
        json.dump(datadict, f) 
        
    return datadict


## Get a datasets including fidelity, count dict of fidelity test, BDM value and BDM score

In [37]:
experiment_id = "exp_04"

n_min=3
n_max=6
datafolder=".\\data\\"
num_of_experiments=100
backend = Aer.get_backend('qasm_simulator') 

run_random_operator_experiment(n_min, n_max, datafolder, num_of_experiments, backend, experiment_id)

{'0': {'op_npy': '.\\data\\0.npy',
  'qnum': 7,
  'fidelity': 0.527,
  'fid_dict': {'1': 527, '0': 473},
  'bdm_val': 3554.763373183966,
  'bdm_score': 0.1720851745967663},
 '1': {'op_npy': '.\\data\\1.npy',
  'qnum': 9,
  'fidelity': 0.478,
  'fid_dict': {'1': 478, '0': 522},
  'bdm_val': 3502.5857117441215,
  'bdm_score': 0.16925194252581657},
 '2': {'op_npy': '.\\data\\2.npy',
  'qnum': 9,
  'fidelity': 0.477,
  'fid_dict': {'1': 477, '0': 523},
  'bdm_val': 3859.1836340013397,
  'bdm_score': 0.1869678398960065},
 '3': {'op_npy': '.\\data\\3.npy',
  'qnum': 9,
  'fidelity': 0.541,
  'fid_dict': {'1': 541, '0': 459},
  'bdm_val': 4851.215740703693,
  'bdm_score': 0.19388442814176443},
 '4': {'op_npy': '.\\data\\4.npy',
  'qnum': 7,
  'fidelity': 0.476,
  'fid_dict': {'1': 476, '0': 524},
  'bdm_val': 3610.1723544009856,
  'bdm_score': 0.1745029416612608},
 '5': {'op_npy': '.\\data\\5.npy',
  'qnum': 9,
  'fidelity': 0.469,
  'fid_dict': {'1': 469, '0': 531},
  'bdm_val': 3271.5935074